In [9]:
from numpy import array
from numpy import hstack
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
import pandas as pd
import numpy as np
import os

In [10]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the dataset
		if end_ix > len(sequences)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [11]:
#Returns array of values with True_data(available frames)
def Missing_Frames(data_info):
    True_Data = np.empty((0, 2), int)
    From = 0
    false_found = 0
    last_true0 = 0
    last_true1 = 0
    for index, row in data_info.iterrows():
        #print(row['Mocap_Data_Exist'])
        if(row['Mocap_Data_Exist'] == False):
            if(false_found == 0):
                True_Data = np.append(True_Data, np.array([[From, index-1]]), axis=0)
            false_found = 1
            From = index+1
            last_true0 = index
        else:
            false_found = 0
            last_true1 = index
    True_Data = np.append(True_Data, np.array([[last_true0+1, last_true1]]), axis=0)
    return True_Data

In [ ]:
def LSTM_Model(X,y,n_steps,n_features):
    #model definition and fit
    model = Sequential()
    model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
    model.add(LSTM(100, activation='relu'))
    model.add(Dense(n_features))
    model.compile(optimizer='adam', loss='mse')
    # fit model
    model.fit(X, y, epochs=400, verbose=0)
    
    #reduce data used to train
    #computing loss?
    #100 epoch -> acceptable loss, 
    #reduce size of neural network - 100 neurons
    #Optimizers - Adam, gradient descent(go to minimum step by step, exp interm of computation) repace with schochastic GD
    #SGD: choose batch size and take decision based on the deicision,
    #Adapter GD (Adam), 

In [12]:
#Timeseries Prediction Function
def PredictTimeSeries(x_input,n_times,n_steps,n_features):
    # demonstrate prediction for next 10 days
    #x_input = array([[ -54,1618,65,-35,1528,-20,-147,1536,83],
     #                [-55,1617,63,-36,1526,-21,147,1535,82], 
      #               [-55,1617,62,-36,1525,-22,148,1535,81], 
       #              [-55,1617,61,-36,1535,-22,-148,1535,80]])

    temp_input=list(x_input)
    lst_output=[]
    i=0
    while(i<n_times):
        #this length is same as steps
        if(len(temp_input)>n_steps):
            #print("temp input 3 ",  temp_input)
            x_input=array(temp_input[1:])
            #print("temp input 4 ",  x_input)
            #print("{} day input {}".format(i,x_input))
            #print(x_input)
            x_input = x_input.reshape((1, n_steps, n_features))
            #print(x_input)
            yhat = model.predict(x_input, verbose=0)
            #print("{} day output {}".format(i,yhat))
            temp_input.append(yhat[0])
            temp_input=temp_input[1:]
            #print(temp_input)
            lst_output.append(yhat[0])
            i=i+1
        else:
            x_input = x_input.reshape((1, n_steps, n_features))
            yhat = model.predict(x_input, verbose=0)
            #print(yhat[0])
            #print("temp input 1 ",  temp_input)
            temp_input.append(yhat[0])
            #print("temp input 2 ",  temp_input)
            lst_output.append(yhat[0])
            i=i+1
    return lst_output

In [13]:
#Get list of info csv form directory
def fn_getfilenames(dir): 
    return os.listdir(dir)

dir="data"
file_list = fn_getfilenames(dir)
infofile_list = []
for file in file_list:
    if "_info.csv" in file:
        infofile_list.append(file)
infofile_list.sort()
infofile_list

['s037t01_info.csv',
 's037t02_info.csv',
 's037t03_info.csv',
 's037t04_info.csv',
 's037t06_info.csv',
 's038t07_info.csv',
 's039t03_info.csv',
 's040t01_info.csv',
 's040t04_info.csv',
 's041t01_info.csv',
 's041t05_info.csv',
 's042t06_info.csv',
 's043t02_info.csv',
 's044t01_info.csv',
 's044t03_info.csv',
 's046t05_info.csv',
 's047t02_info.csv',
 's047t04_info.csv',
 's048t06_info.csv',
 's049t01_info.csv',
 's049t06_info.csv',
 's050t01_info.csv',
 's050t06_info.csv',
 's052t02_info.csv',
 's052t06_info.csv',
 's054eo_info.csv',
 's054t04_info.csv',
 's054t05_info.csv',
 's054t06_info.csv',
 's055eo_info.csv',
 's055t03_info.csv',
 's056t05_info.csv',
 's057t05_info.csv',
 's057t06_info.csv',
 's058t03_info.csv',
 's059t04_info.csv',
 's063t02_info.csv',
 's064t03_info.csv',
 's070t04_info.csv']

In [19]:
# choose a number of time steps for Time Series
n_steps = 2
#5 works well without zeros

In [20]:
#for all Files
print("Total files to process(motion cap): " + str(len(infofile_list)))
predicted_all = pd.DataFrame() #to store the predicted datavalues
file_count=1

for file in infofile_list:
    #missing frames dataset with True/False
    print("\n" + str(file_count) + ".Processing file : " + file + " ...")
    data_info = pd.read_csv(dir + "/" + file)
    TrueFalse_Data = Missing_Frames(data_info)
    
    #print(TrueFalse_Data)
    
    
    mocap_file = file.replace("_info.csv", "_mocap.csv")
    data = pd.read_csv(dir + "/" + mocap_file)
    print("Missing frame(True) summary of this file")
    print(TrueFalse_Data)
    
    #convert as array for data prep(sequencing)
    dataset = data.to_numpy()
    
    #Input only the previous Predicted and True Frame - to model and train/fit
    # Loop For all the missing frame
    rowIndex=0
    #True_Frame = data.iloc[TrueFalse_Data[0][0]:TrueFalse_Data[rowIndex][1]+1,]
    #if 3 entries in TrueFalse_Data, only 2 missing sections
    print("There are " + str(len(TrueFalse_Data)-1) +  " missing frames in the file")
    while(rowIndex < len(TrueFalse_Data)-1):
        #print(TrueFalse_Data[rowIndex][0])
        #print(TrueFalse_Data[rowIndex][1])

        #Aaailable True rows for modeling
        True_Frame = data.iloc[TrueFalse_Data[rowIndex][0]:TrueFalse_Data[rowIndex][1]+1,]
        #if(rowIndex > 0 ):
        #    True_Frame = pd.concat([predicted_df,True_Frame], ignore_index=True)
        #print("True frame for prediction")
        #print(True_Frame.head(50))

        #Convert to Array
        dataset = True_Frame.to_numpy()
        #print(dataset)

        # convert into input/output
        X, y = split_sequences(dataset, n_steps)
        #print(X.shape, y.shape)

        # summarize the data
        #for i in range(len(X)):
         #   print(X[i], y[i])
            #print(X[i])

        #RNN - LSTM
        print("Training LSTM Model ...")
        # the dataset knows the number of features, e.g. 2
        n_features = X.shape[2]
        # define and fit model
        #LSTM_Model(X,y,n_steps,n_features)

        model = Sequential()
        model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(LSTM(100, activation='relu'))
        model.add(Dense(n_features))
        model.compile(optimizer='adam', loss='mse')
        # fit model
        model.fit(X, y, epochs=50, verbose=0)


        # demonstrate prediction
        x_input = X[len(X)-1]
        #print(x_input)
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        #print(yhat)

        print("Predicting Missing frame " + str(rowIndex+1) + " ...")
        #Prediction 
        #print(TrueFalse_Data[rowIndex][1]+1)
        #print(TrueFalse_Data[rowIndex+1][0]-1)
        MissingFramesCount = (TrueFalse_Data[rowIndex+1][0]-1) - (TrueFalse_Data[rowIndex][1])
        lst_output = PredictTimeSeries(x_input,MissingFramesCount,n_steps,n_features)
        #print(round(pd.DataFrame(lst_output)))
        predicted_df = round(pd.DataFrame(lst_output))
        predicted_df.set_axis(['HeadTop_X', 'HeadTop_Y', 'HeadTop_Z', 'HeadFront_X', 'HeadFront_Y','HeadFront_Z','HeadSide_X','HeadSide_Y','HeadSide_Z'], axis='columns', inplace=True)
        #predicted_df.set_axis(data.columns, axis='columns', inplace=True)
        #print(predicted_df)
        predicted_all = pd.concat([predicted_all, predicted_df], ignore_index=True)
        
        #if(rowIndex == 0):
        #    True_Plus_Predicted_Frame = pd.concat([True_Frame, predicted_df], ignore_index=True)
        #else:
        #    True_Plus_Predicted_Frame = pd.concat([True_Plus_Predicted_Frame, True_Frame, predicted_df], ignore_index=True)
        #print("True and predicted df")
        #print(True_Plus_Predicted_Frame)


        #break
        print("Missing frame " + str(rowIndex+1) + " completed.\n")
        rowIndex=rowIndex+1
    #print(predicted_all.shape)
    file_count = file_count + 1
    #Last_Frame = data.iloc[TrueFalse_Data[rowIndex][0]:TrueFalse_Data[rowIndex][1]+1,]
    #True_Plus_Predicted_Frame = pd.concat([True_Plus_Predicted_Frame, Last_Frame], ignore_index=True)
#file_name = "FD_Pred/" + file.replace("_info.csv", "_pred.csv")
file_name = "Pred/bbdc_pred_mocap.csv" 
predicted_all.to_csv(file_name,index=False)
predicted_all.shape

print("\nAll files are Completed")

Total files to process(motion cap): 39

1.Processing file : s037t01_info.csv ...
Missing frame(True) summary of this file
[[   0  300]
 [ 445 1203]
 [1311 1805]
 [1934 2407]
 [2428 2708]
 [2885 3310]
 [3362 3912]
 [4072 4792]]
There are 7 missing frames in the file
Training LSTM Model ...
Predicting Missing frame 1 ...
Missing frame 1 completed.

Training LSTM Model ...
Predicting Missing frame 2 ...
Missing frame 2 completed.

Training LSTM Model ...
Predicting Missing frame 3 ...
Missing frame 3 completed.

Training LSTM Model ...
Predicting Missing frame 4 ...
Missing frame 4 completed.

Training LSTM Model ...
Predicting Missing frame 5 ...
Missing frame 5 completed.

Training LSTM Model ...
Predicting Missing frame 6 ...
Missing frame 6 completed.

Training LSTM Model ...
Predicting Missing frame 7 ...
Missing frame 7 completed.


2.Processing file : s037t02_info.csv ...
Missing frame(True) summary of this file
[[   0  902]
 [ 960 1203]
 [1294 1504]
 [1536 1805]
 [1819 2407]
 [254

Predicting Missing frame 1 ...
Missing frame 1 completed.

Training LSTM Model ...
Predicting Missing frame 2 ...
Missing frame 2 completed.

Training LSTM Model ...
Predicting Missing frame 3 ...
Missing frame 3 completed.

Training LSTM Model ...
Predicting Missing frame 4 ...
Missing frame 4 completed.

Training LSTM Model ...
Predicting Missing frame 5 ...
Missing frame 5 completed.

Training LSTM Model ...
Predicting Missing frame 6 ...
Missing frame 6 completed.

Training LSTM Model ...
Predicting Missing frame 7 ...
Missing frame 7 completed.

Training LSTM Model ...
Predicting Missing frame 8 ...
Missing frame 8 completed.

Training LSTM Model ...
Predicting Missing frame 9 ...
Missing frame 9 completed.


11.Processing file : s041t05_info.csv ...
Missing frame(True) summary of this file
[[   0  300]
 [ 440  902]
 [ 963 1203]
 [1397 1504]
 [1533 2407]
 [2497 3310]
 [3384 3611]
 [3646 4213]
 [4372 4514]
 [4668 5432]]
There are 9 missing frames in the file
Training LSTM Model ...

Missing frame 6 completed.

Training LSTM Model ...
Predicting Missing frame 7 ...
Missing frame 7 completed.


19.Processing file : s048t06_info.csv ...
Missing frame(True) summary of this file
[[   0  601]
 [ 607  902]
 [1039 1805]
 [1958 2407]
 [2495 2708]
 [2861 3009]
 [3141 3825]]
There are 6 missing frames in the file
Training LSTM Model ...
Predicting Missing frame 1 ...
Missing frame 1 completed.

Training LSTM Model ...
Predicting Missing frame 2 ...
Missing frame 2 completed.

Training LSTM Model ...
Predicting Missing frame 3 ...
Missing frame 3 completed.

Training LSTM Model ...
Predicting Missing frame 4 ...
Missing frame 4 completed.

Training LSTM Model ...
Predicting Missing frame 5 ...
Missing frame 5 completed.

Training LSTM Model ...
Predicting Missing frame 6 ...
Missing frame 6 completed.


20.Processing file : s049t01_info.csv ...
Missing frame(True) summary of this file
[[   0  601]
 [ 734  902]
 [1019 1203]
 [1207 1856]]
There are 3 missing frames in the file


Predicting Missing frame 1 ...
Missing frame 1 completed.

Training LSTM Model ...
Predicting Missing frame 2 ...
Missing frame 2 completed.

Training LSTM Model ...
Predicting Missing frame 3 ...
Missing frame 3 completed.

Training LSTM Model ...
Predicting Missing frame 4 ...
Missing frame 4 completed.

Training LSTM Model ...
Predicting Missing frame 5 ...
Missing frame 5 completed.

Training LSTM Model ...
Predicting Missing frame 6 ...
Missing frame 6 completed.

Training LSTM Model ...
Predicting Missing frame 7 ...
Missing frame 7 completed.

Training LSTM Model ...
Predicting Missing frame 8 ...
Missing frame 8 completed.

Training LSTM Model ...
Predicting Missing frame 9 ...
Missing frame 9 completed.

Training LSTM Model ...
Predicting Missing frame 10 ...
Missing frame 10 completed.

Training LSTM Model ...
Predicting Missing frame 11 ...
Missing frame 11 completed.


30.Processing file : s055eo_info.csv ...
Missing frame(True) summary of this file
[[   0  300]
 [ 331  601

In [ ]:
#17.02 - 

In [ ]:
#for all Files
print("Total files to process: " + str(len(infofile_list)))
infofile_list.sort()
for file in infofile_list:
    #missing frames dataset with True/False
    print("\nProcessing file : " + file + " ...")
    data_info = pd.read_csv(dir + "/" + file)
    TrueFalse_Data = Missing_Frames(data_info)
    
    #print(TrueFalse_Data)
    
    
    mocap_file = file.replace("_info.csv", "_mocap.csv")
    data = pd.read_csv(dir + "/" + mocap_file)
    
    #convert as array for data prep(sequencing)
    dataset = data.to_numpy()
    
    #Input only the previous Predicted and True Frame - to model and train/fit
    # Loop For all the missing frame
    rowIndex=0
    #True_Frame = data.iloc[TrueFalse_Data[0][0]:TrueFalse_Data[rowIndex][1]+1,]
    #if 3 entries in TrueFalse_Data, only 2 missing sections
    print("There are " + str(len(TrueFalse_Data)-1) +  " missing frames in the file")
    while(rowIndex < len(TrueFalse_Data)-1):
        #print(TrueFalse_Data[rowIndex][0])
        #print(TrueFalse_Data[rowIndex][1])

        #Aaailable True rows for modeling
        True_Frame = data.iloc[TrueFalse_Data[rowIndex][0]:TrueFalse_Data[rowIndex][1]+1,]
        if(rowIndex > 0 ):
            True_Frame = pd.concat([predicted_df,True_Frame], ignore_index=True)
        #print("True frame for prediction")
        #print(True_Frame.head(50))

        #Convert to Array
        dataset = True_Frame.to_numpy()
        #print(dataset)

        # convert into input/output
        X, y = split_sequences(dataset, n_steps)
        #print(X.shape, y.shape)

        # summarize the data
        #for i in range(len(X)):
         #   print(X[i], y[i])
            #print(X[i])

        #RNN - LSTM
        print("Training LSTIM Model ...")
        # the dataset knows the number of features, e.g. 2
        n_features = X.shape[2]
        # define and fit model
        #LSTM_Model(X,y,n_steps,n_features)

        model = Sequential()
        model.add(LSTM(100, activation='relu', return_sequences=True, input_shape=(n_steps, n_features)))
        model.add(LSTM(100, activation='relu'))
        model.add(Dense(n_features))
        model.compile(optimizer='adam', loss='mse')
        # fit model
        model.fit(X, y, epochs=50, verbose=0)


        # demonstrate prediction
        x_input = X[len(X)-1]
        #print(x_input)
        x_input = x_input.reshape((1, n_steps, n_features))
        yhat = model.predict(x_input, verbose=0)
        #print(yhat)

        print("Predicting Missing frame:  " + str(rowIndex+1) + " ...")
        #Prediction 
        #print(TrueFalse_Data[rowIndex][1]+1)
        #print(TrueFalse_Data[rowIndex+1][0]-1)
        MissingFramesCount = (TrueFalse_Data[rowIndex+1][0]-1) - (TrueFalse_Data[rowIndex][1])
        lst_output = PredictTimeSeries(x_input,MissingFramesCount,n_steps,n_features)
        #print(round(pd.DataFrame(lst_output)))
        predicted_df = round(pd.DataFrame(lst_output))
        predicted_df.set_axis(['HeadTop_X', 'HeadTop_Y', 'HeadTop_Z', 'HeadFront_X', 'HeadFront_Y','HeadFront_Z','HeadSide_X','HeadSide_Y','HeadSide_Z'], axis='columns', inplace=True)
        if(rowIndex == 0):
            True_Plus_Predicted_Frame = pd.concat([True_Frame, predicted_df], ignore_index=True)
        else:
            True_Plus_Predicted_Frame = pd.concat([True_Plus_Predicted_Frame, True_Frame, predicted_df], ignore_index=True)
        #print("True and predicted df")
        #print(True_Plus_Predicted_Frame)


        #break
        rowIndex=rowIndex+1
    Last_Frame = data.iloc[TrueFalse_Data[rowIndex][0]:TrueFalse_Data[rowIndex][1]+1,]
    True_Plus_Predicted_Frame = pd.concat([True_Plus_Predicted_Frame, Last_Frame], ignore_index=True)
    file_name = "Pred/" + file.replace("_info.csv", "_pred.csv")  ###Pred is a folder name to store the output of the predicted values.
    True_Plus_Predicted_Frame.to_csv(file_name,index=False)
    True_Plus_Predicted_Frame.shape

print("\nAll files are Completed")